In [1]:
import pandas as pd
from pandasql import sqldf
import numpy as np
import random 

###### Для решения подобных задач используется синтаксис SQLite
###### Источник, откуда были взяты условия и датасеты всех задач: https://habr.com/ru/company/otus/blog/541882/

## 1. Конкатенация

- Необходимо: 
    * Для каждого лида вывести список тегов, разделенных запятой в одном столбце

In [29]:
# Подготовим данные 
table_of_leads = pd.DataFrame({
        "lead_id": [6, 6, 104, 116, 116, 126, 126],
        "tag": ['fb', 'tilda', 'inst', 'google', 'ecwid', 'tilda', 'organic'],
})

query_2 = """SELECT lead_id,
             GROUP_CONCAT(tag, ', ') AS tags
             FROM table_of_leads
             GROUP BY lead_id
             ORDER BY lead_id DESC"""
sqldf(query_2)

,lead_id,tags
0,126,"tilda, organic"
1,116,"google, ecwid"
2,104,inst
3,6,"fb, tilda"


## 2. Аналитические функции при сохранении всех строк выборки
- Необходимо:
    * Ко всем транзакциям пользователя вывести дату первой покупки
    * К каждой транзакции добавить дату предыдущей транзакции пользователя
    * Показать сумму покупок пользователя нарастающим итогом
    * Присвоить всем транзакциям пользователя / продавца / отделения порядковый номер

In [30]:
table_of_sales = pd.DataFrame({
   'salesid': [30001, 10001, 10005, 40001, 10006, 20001, 40005, 20002, 30003, 30004, 30007,],
   'dateid': ['8/2/2003', '12/24/2003', '12/24/2003', '1/9/2004', '1/18/2004',
              '2/12/2004','2/12/2004','2/16/2004','4/18/2004','4/18/2004','9/7/2004',],
   'sellerid': [3, 1, 1, 4, 1, 2, 4, 2, 3, 3, 3,],
   'buyerid': ['b', 'c', 'a', 'a', 'c', 'b', 'a', 'c', 'b', 'b', 'c',],
   'qty': [10, 10, 30, 40, 10, 20, 10, 20, 15, 20, 30,],
   'qty_shipped': [10, 10, np.nan, np.nan, np.nan, 20, 10, 20, np.nan, np.nan, np.nan,],  
})
table_of_sales['dateid'] = pd.to_datetime(table_of_sales['dateid'])
table_of_sales['dateid'] = table_of_sales['dateid'].dt.date

In [31]:
query_3 = """SELECT salesid,
                    dateid,
                    sellerid,
                    buyerid,
                    qty,
                    FIRST_VALUE(dateid) OVER (PARTITION BY buyerid) AS first_purchase_dt,
                    LAG(dateid) OVER(PARTITION BY buyerid) AS previous_purchase_dt,
                    SUM(qty) OVER (PARTITION BY buyerid ORDER BY dateid) AS moving_qty,
                    ROW_NUMBER() OVER (PARTITION BY buyerid ORDER BY dateid) AS order_number
             FROM table_of_sales
             ORDER BY buyerid ASC
             """

In [32]:
sqldf(query_3)

,salesid,dateid,sellerid,buyerid,qty,first_purchase_dt,previous_purchase_dt,moving_qty,order_number
0,10005,2003-12-24,1,a,30,2003-12-24,None,30,1
1,40001,2004-01-09,4,a,40,2003-12-24,2003-12-24,70,2
2,40005,2004-02-12,4,a,10,2003-12-24,2004-01-09,80,3
3,30001,2003-08-02,3,b,10,2003-08-02,None,10,1
4,20001,2004-02-12,2,b,20,2003-08-02,2003-08-02,30,2
5,30003,2004-04-18,3,b,15,2003-08-02,2004-02-12,65,3
6,30004,2004-04-18,3,b,20,2003-08-02,2004-04-18,65,4
7,10001,2003-12-24,1,c,10,2003-12-24,None,10,1
8,10006,2004-01-18,1,c,10,2003-12-24,2003-12-24,20,2
9,20002,2004-02-16,2,c,20,2003-12-24,2004-01-18,40,3


## 3. Работа с NULL и применение логики ветвления IF-THEN-ELSE в SQL
- Необходимо:
    * Обработать ошибку деления на 0 (divide by zero error)
    * Как выводить NULL (В СЛУЧАЕ С PANDAS SQL - ЭТО  "NONE") вместо пустых строк (‘’)

In [33]:
table_of_leads_2 = pd.DataFrame({
        "lead_id": [6, 6, 7, 104, 115, 116, 116, 126, 126],
        "tag": ['fb', 'tilda', '', 'inst', '', 'google', 'ecwid', 'tilda', 'organic'],
})

In [34]:
query_4 = """SELECT lead_id,
             NULLIF(tag, '') AS tag
             FROM table_of_leads_2"""
sqldf(query_4)

,lead_id,tag
0,6,fb
1,6,tilda
2,7,None
3,104,inst
4,115,None
5,116,google
6,116,ecwid
7,126,tilda
8,126,organic


- Необходимо:
 * Использовать NVL2 для вывода 1 в случае, если она ненулевая и 0 - является нулевой

In [35]:
table_of_transactions = pd.DataFrame({
    "transaction_id": ['5a81a4949e209e0009000002', 
                       '5a81a4969e209e0009000003', 
                       '5a81ae6b9e209e000900000f', 
                       '5ab4ec8433c1e20009000000', 
                       '542425c22cc1b32895000001', 
                       '542425fd2cc1b32895000002', 
                       '5424284a2cc1b3ee3a000026', 
                       '542428f92cc1b32895000012'],
                "ts": ['12-Feb-2018 02:28:36 PM',
                       '12-Feb-2018 02:28:38 PM',
                       '12-Feb-2018 03:10:35 PM',
                       '23-Mar-2018 12:01:08 PM',
                       '25-Sep-2014 02:25:06 PM',
                       '25-Sep-2014 02:26:05 PM',
                       '25-Sep-2014 02:35:54 PM',
                       '25-Sep-2014 02:38:49 PM',],
        "invoice_id": ['58b6b60188d6d9000c00003e',
                       '58b6ba2688d6d9000c00028a',
                       '55bf2d168a5da5e3a60000e7',
                       '58b6b60188d6d9000c00003e',
                        np.nan,
                        np.nan,
                        np.nan,
                        np.nan]
})

In [36]:
# Примечание: т.к. я не использую Oracle 11g R2, эта функция мне недоступна в SQLite, 
# воспользуюсь костыльным способом:
query_5 = """SELECT transaction_id,
                    ts,
                    invoice_id,
                    (CASE WHEN invoice_id NOT NULL THEN 1 ELSE 0 END) AS is_b2b
             FROM table_of_transactions"""
sqldf(query_5)

,transaction_id,ts,invoice_id,is_b2b
0,5a81a4949e209e0009000002,12-Feb-2018 02:28:36 PM,58b6b60188d6d9000c00003e,1
1,5a81a4969e209e0009000003,12-Feb-2018 02:28:38 PM,58b6ba2688d6d9000c00028a,1
2,5a81ae6b9e209e000900000f,12-Feb-2018 03:10:35 PM,55bf2d168a5da5e3a60000e7,1
3,5ab4ec8433c1e20009000000,23-Mar-2018 12:01:08 PM,58b6b60188d6d9000c00003e,1
4,542425c22cc1b32895000001,25-Sep-2014 02:25:06 PM,None,0
5,542425fd2cc1b32895000002,25-Sep-2014 02:26:05 PM,None,0
6,5424284a2cc1b3ee3a000026,25-Sep-2014 02:35:54 PM,None,0
7,542428f92cc1b32895000012,25-Sep-2014 02:38:49 PM,None,0


- Необходимо присвоить численному коду (или, например, битовой маске) текстовые наименования с помощью функции "Decode", для вывода: 
    * "charge", если 0
    * "authorize", если 1, 
    * "settle", если 2

In [37]:
table_of_transactions_2 = pd.DataFrame({
    "transaction_id": [ '542413ba255f00ddb500000c', 
                        '542426162cc1b32895000004',
                        '5424265a2cc1b32895000006',
                        '542426cc2cc1b32895000008',
                        '5424274c2cc1b32895000009',
                        '54242c382cc1b341ea000001',
                        '54242ce22cc1b341ea000008',
                        '54242e242cc1b341ea00000a',
                        '542431782cc1b374b3000076',
                        '542432872cc1b341ea00001c',],
    "status": [0,2,0,2,1,0,0,1,0,1,]
})


In [38]:
# Примечание: т.к. я не использую Oracle 11g R2, эта функция мне недоступна, воспользуюсь костыльным способом:
query_6 = """SELECT transaction_id,
                    (CASE 
                        WHEN status IS 0 THEN "charge"
                        WHEN status IS 2 THEN "settle"
                        ELSE "authorize"
                    END) AS status_modified
             FROM table_of_transactions_2"""

In [39]:
sqldf(query_6)

,transaction_id,status_modified
0,542413ba255f00ddb500000c,charge
1,542426162cc1b32895000004,settle
2,5424265a2cc1b32895000006,charge
3,542426cc2cc1b32895000008,settle
4,5424274c2cc1b32895000009,authorize
5,54242c382cc1b341ea000001,charge
6,54242ce22cc1b341ea000008,charge
7,54242e242cc1b341ea00000a,authorize
8,542431782cc1b374b3000076,charge
9,542432872cc1b341ea00001c,authorize


## 4. Дедупликация данных
- Необходимо: 
    * Выбрать самую актуальную запись с учетом статуса (успешная / отмененная транзакция) и временной метки

In [40]:
table_of_statuses = pd.DataFrame ({
    "transaction_id": ['0004ee71-afa6-71c8-0004-ee7100005f61', 
                       '0004ee71-afa6-71c8-0004-ee7100005f61', 
                       '0004ee71-afa6-71c8-0004-ee7100005f61', 
                       '0004ee71-afa6-71c8-0004-ee7100005f61', 
                       '00089c36-aec0-7b4d-90ca-d2d300005f61', 
                       '00089c36-aec0-7b4d-90ca-d2d300005f61', 
                       '00089c36-aec0-7b4d-90ca-d2d300005f61', 
                       '00089c36-aec0-7b4d-90ca-d2d300005f61', 
                       '00089c36-aec0-7b4d-90ca-d2d300005f61', 
                       '000a6c07-012e-7e43-000a-6c0700005f61', 
                       '000a6c07-012e-7e43-000a-6c0700005f61', 
                       '000a6c07-012e-7e43-000a-6c0700005f61', 
                       '000a6c07-012e-7e43-000a-6c0700005f61', 
                       '000a6c07-012e-7e43-000a-6c0700005f61', 
                       '000a6c07-012e-7e43-000a-6c0700005f61',],
    "is_successful": ['false',
                      'false',
                      'true',
                      'false',
                      'false',
                      'false',
                      'true',
                      'true',
                      'true',
                      'false',
                      'false',
                      'false',
                      'true',
                      'true',
                      'false',],
    "ts": ['24-Mar-2017 10:09:10 AM', 
           '24-Mar-2017 10:15:41 AM', 
           '24-Mar-2017 10:21:49 AM', 
           '24-Mar-2017 10:21:49 AM', 
           '25-Mar-2017 12:58:00 PM', 
           '25-Mar-2017 01:00:39 PM', 
           '25-Mar-2017 01:09:42 PM', 
           '25-Mar-2017 01:12:42 PM', 
           '25-Mar-2017 02:06:20 PM', 
           '25-Mar-2017 02:33:02 PM', 
           '25-Mar-2017 02:56:24 PM', 
           '25-Mar-2017 03:36:57 PM', 
           '25-Mar-2017 03:42:47 PM', 
           '25-Mar-2017 03:58:17 PM', 
           '25-Mar-2017 04:11:07 PM'],
})

In [41]:
query_7 = """SELECT query.transaction_id, query.is_successful, query.ts
                FROM
                (SELECT 
                 transaction_id,
                 is_successful,
                 ts,
                 MAX(ts) as actual_date
                     FROM 
                         table_of_statuses
                         WHERE is_successful = 'true'
                         GROUP BY transaction_id) query
             """
sqldf(query_7)

,transaction_id,is_successful,ts
0,0004ee71-afa6-71c8-0004-ee7100005f61,true,24-Mar-2017 10:21:49 AM
1,00089c36-aec0-7b4d-90ca-d2d300005f61,true,25-Mar-2017 02:06:20 PM
2,000a6c07-012e-7e43-000a-6c0700005f61,true,25-Mar-2017 03:58:17 PM


## 5. Анализ временных рядов
- Необходимо
    * Получить текущей даты (+ время) - CURRENTDATE, CURRENTTIMESTAMP
    * Прошедших дней до события 
    * ~Подсчет времени истечения срока действия события - DATEADD~
    * ~Дата начала недели, в которой произошло событие - DATETRUNC~ Костыли не работают
    * Конвертация Unix Timestamp (epoch) в человекочитаемый формат

In [42]:
table_of_dates = pd.DataFrame({
    "ts": ["2022-02-04T21:39:05Z",
           "2022-02-05T21:35:00Z",
           "2022-01-09T21:29:29Z",
           "2022-02-09T21:29:21Z",
           "2022-02-02T21:00:26Z",
           "2022-02-15T21:26:10Z",
           "2022-01-09T21:39:05Z",
           "2022-02-01T21:24:11Z",
           "2022-02-02T21:19:46Z",],
    "metadata_ts_epoch": [1612906751860,
                          1612906553854,
                          1612906193643,
                          1612906192997,
                          1612906101791,
                          1612906008959,
                          1612905962959,
                          1612905613963,
                          1612905594345,],
})
table_of_dates['ts'] = pd.to_datetime(table_of_dates['ts'], format='%Y-%m-%dT%H:%M:%SZ')

In [43]:
query_8 = """SELECT DATETIME(ts) AS ts, 
                    metadata_ts_epoch, 
                    CURRENT_DATE AS current_dt, 
                    CURRENT_TIMESTAMP current_ts, 
                    CAST(julianday() - julianday(DATE(ts)) AS INTEGER) days_since_ts,
                    DATETIME(ROUND(metadata_ts_epoch / 1000), 'unixepoch') AS converted_metadata_ts
                    FROM table_of_dates"""
sqldf(query_8)

,ts,metadata_ts_epoch,current_dt,current_ts,days_since_ts,converted_metadata_ts
0,2022-02-04 21:39:05,1612906751860,2022-03-19,2022-03-19 18:09:03,43,2021-02-09 21:39:11
1,2022-02-05 21:35:00,1612906553854,2022-03-19,2022-03-19 18:09:03,42,2021-02-09 21:35:53
2,2022-01-09 21:29:29,1612906193643,2022-03-19,2022-03-19 18:09:03,69,2021-02-09 21:29:53
3,2022-02-09 21:29:21,1612906192997,2022-03-19,2022-03-19 18:09:03,38,2021-02-09 21:29:52
4,2022-02-02 21:00:26,1612906101791,2022-03-19,2022-03-19 18:09:03,45,2021-02-09 21:28:21
5,2022-02-15 21:26:10,1612906008959,2022-03-19,2022-03-19 18:09:03,32,2021-02-09 21:26:48
6,2022-01-09 21:39:05,1612905962959,2022-03-19,2022-03-19 18:09:03,69,2021-02-09 21:26:02
7,2022-02-01 21:24:11,1612905613963,2022-03-19,2022-03-19 18:09:03,46,2021-02-09 21:20:13
8,2022-02-02 21:19:46,1612905594345,2022-03-19,2022-03-19 18:09:03,45,2021-02-09 21:19:54


## 6. Анализ истории со Slowly Changing Dimensions (SCD)
- Необходимо:
    * Найти статус был у клиентов на 3-й день месяца? Вывести две колонки client_id, status

In [44]:
table_of_client = pd.DataFrame({
    'client_id': ['510bb1052cc1b36f4a000001',
                  '510bb1052cc1b36f4a000001', 
                  '510bb1052cc1b36f4a000001',
                  '510bb1052cc1b36f4a000053',
                  '510bb1052cc1b36f4a000053',
                  '510bb1052cc1b36f4a000053',
                  '510bb1052cc1b36f4a000053',
                  '510bb1052cc1b36f4a000053',
                  '520bb1052cc1b36f4a000064f',
                  '520bb1052cc1b36f4a000064f',
                  '520bb1052cc1b36f4a000064f',
                  '520bb1052cc1b36f4a000034c2',
                  '520bb1052cc1b36f4a000034c2',
                  '520bb1052cc1b36f4a000034c2',
                  '520bb1052cc1b36f4a0000ddf',
                  '520bb1052cc1b36f4a0000ddf',],
    'status': ['active',
               'has_debts',
               'deactivated',
               'active',
               'has_debts',
               'active',
               'has_debts',
               'active',
               'active',
               'has_debts', 
               'active',
                'active',
               'has_debts',
               'active',
               'active',
               'has_debts',],
    'valid_from': ['2021-02-01T00:00:00Z',
                   '2021-02-03T00:00:00Z',
                   '2021-02-06T00:00:00Z',
                   '2021-02-01T00:00:00Z',
                   '2021-02-01T00:00:00Z',
                   '2021-02-02T00:00:00Z',
                   '2021-02-03T00:00:00Z',
                   '2021-02-04T00:00:00Z',
                   '2021-02-01T00:00:00Z',
                   '2021-02-03T00:00:00Z',
                   '2021-02-05T00:00:00Z',
                   '2021-02-01T00:00:00Z',
                   '2021-02-01T00:00:00Z',
                   '2021-02-02T00:00:00Z',
                   '2021-02-01T00:00:00Z',
                   '2021-02-04T00:00:00Z',
                  ],
    'valid_to': ['2021-02-03',
                 '2021-02-06',
                  np.NAN,
                 '2021-02-01',
                 '2021-02-02',
                 '2021-02-03',
                 '2021-02-04',
                  np.NAN,
                 '2021-02-03',
                 '2021-02-05',
                  np.NAN,
                 '2021-02-01',
                 '2021-02-02',
                  np.NAN,
                 '2021-02-04',
                  np.NAN,
                ]
})

In [45]:
query_9 = """SELECT  client_id,
                     status
                     FROM table_of_client
                     WHERE DATE(valid_from) <= '2021-02-03' and coalesce(DATE(valid_to), date('now')) > '2021-02-03'"""
sqldf(query_9)

,client_id,status
0,510bb1052cc1b36f4a000001,has_debts
1,510bb1052cc1b36f4a000053,has_debts
2,520bb1052cc1b36f4a000064f,has_debts
3,520bb1052cc1b36f4a000034c2,active
4,520bb1052cc1b36f4a0000ddf,active


- Необходимо узнать:
    * Как в течение недели росло количество активных клиентов?

In [46]:
table_of_leads = pd.DataFrame({
    'lead_id': [10868761, 
                10890135,  
                10878901,  
                10898407,  
                10861821,  
                10868163,  
                10873235,  
                10874443,  
                10877471,  
                10896027,  
                10897779,  
                10865963,  
                10877513,  
                10884015,  
                10873059,  
                10868087,  
                10865885,  
                10849427,  
                10857555,  
                10832093,],
    'dt': ['2021-02-02 00:00:00', 
           '2021-02-02 00:00:00', 
           '2021-02-02 00:00:00', 
           '2021-02-02 00:00:00', 
           '2021-02-02 00:00:00', 
           '2021-02-03 00:00:00', 
           '2021-02-03 00:00:00', 
           '2021-02-03 00:00:00', 
           '2021-02-03 00:00:00', 
           '2021-02-03 00:00:00', 
           '2021-02-03 00:00:00', 
           '2021-02-03 00:00:00', 
           '2021-02-04 00:00:00', 
           '2021-02-04 00:00:00', 
           '2021-02-04 00:00:00', 
           '2021-02-04 00:00:00', 
           '2021-02-04 00:00:00', 
           '2021-02-04 00:00:00', 
           '2021-02-04 00:00:00', 
           '2021-02-04 00:00:00',],
    'status': ['ongoing', 
               'success', 
               'cancelled', 
               'success', 
               'ongoing', 
               'cancelled', 
               'success', 
               'success', 
               'cancelled', 
               'success', 
               'ongoing', 
               'success', 
               'cancelled', 
               'success', 
               'cancelled', 
               'success', 
               'success', 
               'ongoing', 
               'success', 
               'success', ],
    'tags': ['facebook tilda sql', 
             'google ecwid python', 
             'facebook tilda sql', 
             'facebook tilda python', 
             'google ecwid go', 
             'facebook tilda python', 
             'facebook ecwid sql',  
             'facebook tilda python', 
             'google ecwid go', 
             'facebook tilda sql', 
             'facebook tilda python', 
             'google ecwid go', 
             'facebook tilda python', 
             'facebook tilda python', 
             'facebook tilda go', 
             'facebook ecwid sql', 
             'google tilda python', 
             'facebook tilda go', 
             'google tilda python', 
             'facebook tilda sql',],
     'amount': [1000, 
                3000, 
                2000, 
                1000, 
                1500, 
                7000, 
                3000, 
                4500, 
                1650, 
                1000, 
                1000, 
                3000, 
                3000, 
                7000, 
                1400, 
                5000, 
                2300, 
                1000, 
                8000, 
                1000,]
})

In [47]:
query_10 = """SELECT dt,
                     leads_success,
                     leads_python,
                     amount_total,
                     amount_python
              FROM (
                  SELECT dt,
                         COUNT(lead_id) AS leads_total,
                         SUM(CASE WHEN status LIKE 's%'       THEN 1      ELSE 0 END) AS leads_success,
                         SUM(CASE WHEN tags   LIKE '%python%' THEN 1      ELSE 0 END) AS leads_python,
                         SUM(amount) AS amount_total,
                         SUM(CASE WHEN tags   LIKE '%python%' THEN amount ELSE 0 END) AS amount_python   
                  FROM table_of_leads
                  GROUP BY dt);"""
sqldf(query_10)

,dt,leads_success,leads_python,amount_total,amount_python
0,2021-02-02 00:00:00,2,2,8500,4000
1,2021-02-03 00:00:00,4,3,21150,12500
2,2021-02-04 00:00:00,5,4,28700,20300
